# Introduction to Pandas: Data Organization and Analysis

So far, we have been working with numerical data using NumPy arrays. But what happens when your data is more complex — when it has column headers, mixed types (numbers, categories, text), or comes from a spreadsheet or CSV file with metadata? For these situations, Python has a powerful library called **pandas**.

Before we dive in, let's briefly step back and think about **data formats** more generally.

## A Note About Data Formats

All data files on your computer are ultimately stored in one of two broad categories:

**Human-readable (text) formats:**
- Can be opened and inspected with any text editor
- Easy to write to and edit by hand
- Examples: `.csv`, `.txt`, `.json`, `.xml`
- Downside: *large and slow* — storing numbers as text is inefficient

**Binary formats:**
- Stored in a compact, machine-readable encoding
- Faster to read/write, can be compressed, support more features
- Examples: `.npy`/`.npz` (NumPy), `.mat` (MATLAB), **HDF5**, **netCDF**
- Downside: not human-readable; you need specific software to open them

Let's start with the basic tools for reading and writing numerical data files, then work our way up to pandas.

In [ ]:
import numpy as np
import scipy.io as io

## Reading Simple Numerical Data with NumPy

If your data is a purely numerical CSV — no header rows, no mixed types, just numbers — `np.loadtxt` is the quickest tool. It reads the file directly into a NumPy array:

In [ ]:
data_filename = "01HIVseries/HIVseries.csv"

# delimiter=',' tells NumPy that columns are separated by commas
data_set = np.loadtxt(data_filename, delimiter=',')
print(data_set)

### MATLAB .mat files

MATLAB `.mat` files can be read with `scipy.io.loadmat`. The result is a Python **dictionary** whose keys are the variable names from MATLAB and whose values are the corresponding arrays.

> **Important note about versions:** After version 7.2, MATLAB switched to an HDF5-based format for `.mat` files. `scipy.io` can only read version 7.2 and earlier. If you are saving a `.mat` file in MATLAB and want to open it in Python with `scipy.io`, use the command `save('data.mat', '-v7')` in MATLAB. For newer `.mat` files you would need the `h5py` package instead.

In [ ]:
# loadmat returns a dictionary -- keys are the variable names from MATLAB
mat_data = io.loadmat(data_filename[:-4] + '.mat')
print(mat_data)  # note the metadata keys __header__, __version__, __globals__

In [ ]:
# The HIV data was stored in a variable called "a" in MATLAB
hiv_from_mat = mat_data['a']
print(hiv_from_mat)

`scipy.io` can also **write** `.mat` files, so you can create data in Python and open it in MATLAB:

```python
io.savemat('mydata.mat', {'x': x, 'y': y})
```

### NumPy's Native Binary Format: .npy and .npz

NumPy has its own binary file format for saving and loading arrays quickly. It is *not* human-readable — you cannot open it in a text editor — so use it only when you know you'll be working entirely in Python. The upside is speed and exact preservation of dtype (Python data-type).

- `np.save('filename', array)` — saves a single array to a `.npy` file
- `np.savez('filename', name1=arr1, name2=arr2)` — saves multiple arrays together into a `.npz` (zipped) file; the keyword argument names become the keys you use to retrieve the arrays
- `np.load('filename.npy')` — loads a single array back
- `np.load('filename.npz')` — loads a `.npz` file into an `NpzFile` object, which works like a dictionary

In [ ]:
x = np.linspace(0, 1, 1001)
y = 3*np.sin(x)**3 - np.sin(x)

# Save individually as .npy files
np.save('x_values', x)
np.save('y_values', y)

# Or save both together in a single .npz file
# The keyword argument names (x_vals, y_vals) become the dictionary keys
np.savez('xy_values', x_vals=x, y_vals=y)

In [ ]:
# Load a single .npy file -- gives back an array directly
x_loaded = np.load('x_values.npy')

# Load a .npz file -- gives back an NpzFile object, not an array directly
xydata = np.load('xy_values.npz')
print('Variables stored in the npz file:', xydata.files)

# Access individual arrays using the keyword names as dictionary keys
x_from_npz = xydata['x_vals']
y_from_npz = xydata['y_vals']
print(x_from_npz)
print(y_from_npz)

Now we have seen the basic toolkit for reading and writing numerical data. But what if your data has column headers, mixed types, or comes from a spreadsheet? That's where **pandas** comes in.

## Why Use Pandas?

The pandas library is designed to make working with labeled, structured data easy and fast. Here is what it brings to the table:

- **Stores data of different types in the same object** — a column of integers next to a column of strings next to a column of dates, all in one structure
- **Labels your data** — rows and columns have names, not just integer indices
- **Easy save/load from popular file types** — CSV, Excel, HDF5, and more, all with one line of code
- **Great organization** — built-in tools for sorting, filtering, and summarizing
- **Easy to compute statistics** — mean, std, describe, groupby, and more
- **Built-in plotting** — quick visualizations directly from a DataFrame

We import pandas with the conventional alias `pd`:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## The Two Core Data Structures: Series and DataFrame

Pandas is built around two objects:

- A **`Series`** is a one-dimensional array with labels (an *index*). Think of it as a single column of data.
- A **`DataFrame`** is a two-dimensional table with labeled rows and labeled columns. Think of it as a spreadsheet or a data matrix where **each column is a Series**.

Both of these are *mutable* data types.

### Creating a Series

You can create a `Series` from a list. Pandas assigns integer indices (0, 1, 2, ...) by default, but notice that `np.nan` is allowed as a missing value:

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
print(s)

### Creating a DataFrame from a Dictionary

One of the most common ways to create a `DataFrame` is from a Python dictionary. The keys become the column names, and the values become the columns. **All columns should have the same length** — if you provide a single scalar value, pandas will repeat it to fill the column.

In [ ]:
# Create a simple DataFrame with election results data
# Notice that we can mix different data types across columns
data = {
    'State': ['TN', 'TN', 'AL', 'AL', 'GA'],
    'District': [1, 2, 1, 2, 1],
    'Dem_2020': [35.1, 41.2, 30.5, 38.7, 48.2],
    'Rep_2020': [62.4, 56.8, 67.9, 59.8, 49.5],
    'Competitive': [False, False, False, False, True]
}

df = pd.DataFrame(data)
df

Notice how the DataFrame displays nicely as a table with column names and an integer row index (0, 1, 2, ...) on the left. You can also check the data types of each column:

In [ ]:
# Each column has its own dtype
print(df.dtypes)

### Creating a DataFrame from a NumPy Array

You can also create a DataFrame from a NumPy array and supply column names and/or an index (row labels). Here we'll use `pd.date_range` to create a date-based index, which is common when working with time series data:

In [ ]:
# Create a date index: 6 consecutive days starting 2013-01-01
dates = pd.date_range('20130101', periods=6)
print(dates)

In [ ]:
# Create a 6x4 DataFrame of random data with the date index
# and column labels A, B, C, D
df_ts = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
df_ts

**Exercise:** In the cell below, create a DataFrame from a dictionary that represents the following small dataset about three hypothetical SIR model runs. Each run has a `beta` parameter, a `gamma` parameter, a computed basic reproduction number `R0` (recall $R_0 = \beta/\gamma$), and a boolean `epidemic` column that is `True` when $R_0 > 1$. Use $\beta$ values 0.3, 0.5, 0.8 and $\gamma$ values 0.4, 0.4, 0.3. Compute `R0` and `epidemic` using Python — don't type the values in by hand.

In [ ]:
beta = np.array([0.3, 0.5, 0.8])
gamma = np.array([0.4, 0.4, 0.3])
R0 = beta / gamma

sir_runs = pd.DataFrame({
    'beta': beta,
    'gamma': gamma,
    'R0': R0,
    'epidemic': R0 > 1
})
sir_runs

## Viewing Data

When you have a large dataset, you won't want to print the whole thing. Pandas gives you several tools to quickly inspect a DataFrame:

- `df.head(n)` — shows the first `n` rows (default 5)
- `df.tail(n)` — shows the last `n` rows (default 5)
- `df.index` — shows the row labels
- `df.columns` — shows the column labels
- `df.describe()` — shows a quick summary of statistics for numerical columns
- `df.T` — transposes the DataFrame (swaps rows and columns)

In [ ]:
# Load the congressional elections data
elections = pd.read_csv('Daily_Kos_Elections_08_12_16_congress_districts.csv')

# Show just the first 5 rows
elections.head()

In [ ]:
# Show the last 3 rows
elections.tail(3)

In [ ]:
# View the index and column labels
print('Index:', elections.index)
print('Columns:', elections.columns.tolist())

In [ ]:
# Quick statistical summary of all numerical columns
elections.describe()

## Getting Data Out: DataFrame.to_numpy()

One of the most important things to know is **how to get your data back out into a NumPy array** so you can do mathematics with it. The method `DataFrame.to_numpy()` does this:

> **Important:** NumPy arrays have *one* dtype for the entire array, while pandas DataFrames have one dtype *per column*. When you call `to_numpy()`, pandas must find a single dtype that can hold all the columns. If your DataFrame has mixed types (e.g., floats and strings), you'll end up with dtype `object`, which is just a Python object array — not useful for math!

In [ ]:
# df_ts is all floats, so to_numpy() is fast and clean
arr = df_ts.to_numpy()
print(type(arr))
print(arr.dtype)
print(arr)

In [ ]:
# For a DataFrame with mixed types (like elections), the result is dtype=object
# Note: to_numpy() does NOT include the index or column labels
arr_elec = elections.to_numpy()
print(arr_elec.dtype)
print(arr_elec[:3])  # first 3 rows

When you only need specific numerical columns for computation, it's better to select just those columns before calling `to_numpy()`. We'll see how to do that in the next section.

## Selecting Subsets of Data

This is one of the most important — and most confusing — aspects of pandas. There are several ways to select data, and they behave differently. Let's go through them carefully.

We'll use `df_ts` (the random date-indexed DataFrame) for most of these examples. If it's been a while, run the cell below to recreate it:

In [ ]:
# Recreate df_ts with a fixed random seed so everyone gets the same data
np.random.seed(42)
dates = pd.date_range('20130101', periods=6)
df_ts = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
df_ts

### Rule 1: Single `[]` with a string selects a **column** (returns a Series)

When you use `df['column_name']`, you get that column as a `Series`. You can also use `df.column_name` as a shorthand (but `[]` notation is safer because it works even when the column name conflicts with a pandas method name).

In [ ]:
# Select a single column -- gives back a Series
col_A = df_ts['A']
print(type(col_A))
print(col_A)

### Rule 2: Single `[]` with a **slice** selects **rows**

When you use `df[0:3]` or `df['20130102':'20130104']`, you get rows. Note: this is the *opposite* of Rule 1! Slicing with `[]` gives rows, not columns. This inconsistency is a genuine gotcha — which is why the `.loc` and `.iloc` methods (below) are preferred.

The idea is that you typically want to select single columns but multiple rows, but of course this isn't always the case.

In [ ]:
# Slicing by integer position gives rows
print(df_ts[0:3])  # first 3 rows

In [ ]:
# Slicing by label gives rows (and both endpoints are INCLUDED)
# THIS WILL BE A GENERAL RULE: slicing by label includes the endpoint, while slicing by integer position does NOT include the endpoint.
print(df_ts['20130102':'20130104'])

### Rule 3: `.loc[]` — selection by **label**

`.loc[row_label, column_label]` selects data by the *name* of the row and column. This is the preferred method when your index has meaningful labels (like dates or strings).

> **Note:** With `.loc`, label slicing **includes both endpoints** (unlike Python slices which exclude the right endpoint).

In [ ]:
# Get a single row by its label (returns a Series)
df_ts.loc[dates[0]]

In [ ]:
# Select all rows (:) but only columns A and B
df_ts.loc[:, ['A', 'B']]

In [ ]:
# Slice rows by label AND select specific columns
# Note: both date endpoints are included!
df_ts.loc['20130102':'20130104', ['A', 'B']]

In [ ]:
# Get a single scalar value by row and column label
df_ts.loc[dates[0], 'A']

### Rule 4: `.iloc[]` — selection by **integer position**

`.iloc[row_position, column_position]` selects data by *integer index*, just like NumPy array indexing. Positions are 0-based, and slices **exclude** the right endpoint (standard Python behavior).

In [ ]:
# Row at integer position 3 (the 4th row)
df_ts.iloc[3]

In [ ]:
# Rows 3 and 4 (positions 3:5), columns 0 and 1 (positions 0:2)
# Right endpoint is EXCLUDED, just like NumPy
df_ts.iloc[3:5, 0:2]

In [ ]:
# Select specific row and column positions using lists
df_ts.iloc[[1, 2, 4], [0, 2]]

In [ ]:
# Get a single scalar by position
df_ts.iloc[1, 1]

### Summary: When to use what

| Syntax | What it does | Notes |
|---|---|---|
| `df['col']` | Select a column (→ Series) | Use for single column |
| `df[0:3]` | Slice rows by position | Use for a block of rows |
| `df.loc[row, col]` | Select by **label** | Both endpoints included in slices |
| `df.iloc[row, col]` | Select by **integer position** | Excludes right endpoint (like NumPy) |

**Exercise:** Using the `elections` DataFrame and `.loc` or `.iloc`, answer the following:

1. In the cell below, select only the `'Clinton\r\n2016'` and `'Trump\r\n2016'` columns for all rows. Save the result to a variable and print the first 5 rows.
2. Then convert it to a NumPy array and compute the mean vote share for each candidate across all districts.

In [ ]:
# Part 1: select two columns
# Note: the column names contain a carriage return + newline (\r\n) from the CSV header
clinton_trump = elections[['Clinton\r\n2016', 'Trump\r\n2016']]
print(clinton_trump.head())

In [ ]:
# Part 2: convert to numpy and compute means
ct_arr = clinton_trump.to_numpy()
print('Mean Clinton 2016 vote share: {:.2f}%'.format(ct_arr[:, 0].mean()))
print('Mean Trump 2016 vote share:   {:.2f}%'.format(ct_arr[:, 1].mean()))

### Boolean Indexing

Just like NumPy, you can use a boolean condition to filter rows. This is extremely useful for finding data that satisfies some criterion.

In [ ]:
# Get all rows where column A is positive
df_ts[df_ts['A'] > 0]

In [ ]:
# Apply a condition across the whole DataFrame:
# values that do NOT satisfy the condition become NaN
df_ts[df_ts > 0]

In [ ]:
# Working with the elections data:
# Find all districts where Clinton got more than 50% of the vote in 2016
clinton_majority = elections[elections['Clinton\r\n2016'] > 50]
print(f"Number of districts where Clinton got >50%: {len(clinton_majority)}")
clinton_majority.head()

**Exercise:** In the cell below, use boolean indexing on the `elections` DataFrame to find all districts where the Republican candidate got more than 70% of the vote in 2016. How many such districts are there? Print the number.

In [ ]:
gop_landslide = elections[elections['Trump\r\n2016'] > 70]
print(f"Districts where Trump got >70%: {len(gop_landslide)}")

## Sorting

You can sort a DataFrame by the values in one or more columns using `sort_values()`, or by the index using `sort_index()`.

In [ ]:
# Sort the elections data by Clinton's 2016 vote share, highest first
elections.sort_values(by='Clinton\r\n2016', ascending=False).head(10)

In [ ]:
# Sort df_ts by column B
df_ts.sort_values(by='B')

In [ ]:
# Sort by index (rows) in reverse order
df_ts.sort_index(ascending=False)

## Operations and Statistics

Pandas makes it easy to compute descriptive statistics. By default, operations **skip missing values** (`NaN`). Operations apply column-by-column (i.e., down each column).

In [ ]:
# Mean of each column
print('Column means:')
print(df_ts.mean())

print('\nRow means (axis=1 means operate along columns, i.e., across each row):')
print(df_ts.mean(axis=1))

In [ ]:
# Comprehensive summary statistics
df_ts.describe()

In [ ]:
# You can apply numpy functions too
print('Standard deviation of each column:')
print(df_ts.std())

**Exercise:** Use the `elections` DataFrame to answer the following:

1. What was the mean vote share for Obama in 2012 across all congressional districts?
2. What was the standard deviation?
3. In how many districts did Obama get more than his mean vote share? (Use boolean indexing.)

In [ ]:
obama_mean = elections['Obama\r\n2012'].mean()
obama_std  = elections['Obama\r\n2012'].std()
print(f"Mean Obama 2012 vote share:  {obama_mean:.2f}%")
print(f"Std dev:                     {obama_std:.2f}%")

above_mean = elections[elections['Obama\r\n2012'] > obama_mean]
print(f"Districts above mean:        {len(above_mean)}")

## Missing Data

Pandas represents missing data as `np.nan`. Many operations ignore missing data by default. There are several tools for dealing with it:

- `df.dropna(how='any')` — drop any row that has at least one `NaN`
- `df.dropna(how='all')` — drop rows where *all* values are `NaN`
- `df.fillna(value)` — fill `NaN` values with a specified value
- `pd.isna(df)` — return a boolean DataFrame showing where the `NaN`s are

In [ ]:
# Create a DataFrame with some missing values for demonstration
df_missing = df_ts.copy()
df_missing.iloc[0, 2] = np.nan   # row 0, column C
df_missing.iloc[3, 0] = np.nan   # row 3, column A
df_missing.iloc[5, :] = np.nan   # all of row 5
df_missing

In [ ]:
# See where the NaNs are
pd.isna(df_missing)

In [ ]:
# Drop any row with at least one NaN
df_missing.dropna(how='any')

In [ ]:
# Fill NaN with 0
df_missing.fillna(value=0)

## Plotting

Pandas DataFrames and Series have a built-in `.plot()` method that calls matplotlib under the hood. It automatically uses column names as labels, which is very convenient.

In [ ]:
# Plot a cumulative sum random walk (one column = one line)
df_walk = pd.DataFrame(
    np.random.randn(200, 4),
    columns=['A', 'B', 'C', 'D']
).cumsum()

plt.figure()
df_walk.plot()
plt.title('Random Walks')
plt.xlabel('Step')
plt.ylabel('Position')
plt.legend(loc='best')
plt.show()

In [ ]:
# Histogram of Clinton vote share across all districts
plt.figure()
elections['Clinton\r\n2016'].plot(kind='hist', bins=30, edgecolor='black')
plt.xlabel('Vote share (%)')
plt.title('Clinton 2016 vote share by congressional district')
plt.show()

**Exercise:** Make a scatter plot of Clinton's 2016 vote share (x-axis) vs. Obama's 2012 vote share (y-axis). Use `kind='scatter'` and set appropriate axis labels. What pattern do you see? Does it make sense?

In [ ]:
plt.figure()
elections.plot(kind='scatter', x='Clinton\r\n2016', y='Obama\r\n2012', alpha=0.4)
plt.xlabel("Clinton 2016 vote share (%)")
plt.ylabel("Obama 2012 vote share (%)")
plt.title("Clinton 2016 vs. Obama 2012 by congressional district")
plt.tight_layout()
plt.show()
# The two are strongly positively correlated -- districts that leaned Democratic
# in 2012 generally stayed Democratic in 2016, though Clinton underperformed
# Obama in many districts (the cloud sits slightly below the diagonal).

## Importing and Exporting Data

One of pandas' greatest strengths is the ease with which you can read and write data in a variety of formats. The general pattern is:

- **Read:** `pd.read_csv()`, `pd.read_excel()`, `pd.read_hdf()`, `pd.read_json()`, ...
- **Write:** `df.to_csv()`, `df.to_excel()`, `df.to_hdf()`, `df.to_json()`, ...

### CSV

In [ ]:
# Write df_ts to a CSV file
df_ts.to_csv('demo_output.csv')

# Read it back
df_reload = pd.read_csv('demo_output.csv', index_col=0)
df_reload

The `index_col=0` argument tells pandas that the first column of the CSV is the row index, not a data column. Without it, you'd get an extra unnamed column.

### HDF5

HDF5 is an efficient binary format well-suited for large datasets. It requires the `tables` package (`conda install pytables`).

In [ ]:
# Write to HDF5 (requires pytables / tables package)
# df_ts.to_hdf('demo.h5', key='timeseries')
# pd.read_hdf('demo.h5', 'timeseries')

# We'll skip the actual execution here since pytables may not be installed,
# but the syntax is exactly the same pattern as CSV.

### Excel

In [ ]:
# Write to Excel (requires openpyxl: conda install openpyxl)
# df_ts.to_excel('demo.xlsx', sheet_name='Sheet1')
# pd.read_excel('demo.xlsx', 'Sheet1', index_col=None, na_values=['NA'])

# Again, same pattern — just a different function name and file extension.

### Reading the HIV Data Again — Now with Pandas

Earlier we loaded the HIV CSV with `np.loadtxt`. Pandas can read the same file and add column labels automatically, making the data much easier to work with:

In [ ]:
# Read the HIV data with pandas and add meaningful column names
hiv = pd.read_csv('01HIVseries/HIVseries.csv', header=None,
                  names=['time_years', 'viral_load'])
hiv

In [ ]:
# Easy to plot directly from the DataFrame
plt.figure()
plt.plot(hiv['time_years'], hiv['viral_load'], 'o-')
plt.xlabel('Time (years)')
plt.ylabel('Viral load')
plt.title('HIV Viral Load Over Time')
plt.show()

In [ ]:
# To get the data out as a numpy array for math:
t = hiv['time_years'].to_numpy()
V = hiv['viral_load'].to_numpy()
print('Time array:', t)
print('Max viral load:', V.max())

## Gotchas: The Truth Value of a Series

Here is a common error that you will almost certainly encounter. Suppose you try to use a boolean condition on an entire array or Series inside an `if` statement:

In [ ]:
# This causes the same kind of error in plain NumPy:
A = np.random.rand(8)
# Uncomment the line below to see the error:
# if A > 0.5:
#     print('bigger')

The error message you get is:

```
ValueError: The truth value of an array is ambiguous. Use a.any() or a.all()
```

The problem is that `A > 0.5` produces a boolean **array**, not a single `True` or `False`. Python's `if` statement needs a single boolean. The fix is to use `.any()` (at least one element satisfies the condition) or `.all()` (all elements satisfy the condition):

In [ ]:
A = np.random.rand(8)
print('A =', A)

# Correct: are ANY elements bigger than 0.5?
if (A > 0.5).any():
    print('At least one element is bigger than 0.5')

# Correct: are ALL elements bigger than 0.5?
if (A > 0.5).all():
    print('All elements are bigger than 0.5')
else:
    print('Not all elements are bigger than 0.5')

The same fix works for a pandas Series or DataFrame. This is one of the most common errors beginners encounter in both NumPy and pandas — now you know how to handle it!

Similarly, when you want to combine two boolean conditions in pandas, use `&` (and) or `|` (or) instead of `and`/`or`, and **wrap each condition in parentheses**. This operates elementwise instead of returning an error due to the fact that `and`/`or` are expecting single values on each side.

In [ ]:
# Districts where Clinton got >45% AND Trump got <50%
# Each condition MUST be in parentheses when combining with & or |
close_races = elections[(elections['Clinton\r\n2016'] > 45) & (elections['Trump\r\n2016'] < 50)]
print(f"Close races (Clinton > 45% and Trump < 50%): {len(close_races)} districts")
close_races.head()

## Putting It All Together

Here is a small worked example that ties together many of the concepts from this notebook. We will use the congressional elections data to look at how Democratic vote share changed from 2008 to 2016.

In [ ]:
# Make a copy so we don't modify the original
df_analysis = elections.copy()

# Add a new column: change in Democratic vote share from 2008 to 2016
df_analysis['Dem_change'] = df_analysis['Clinton\r\n2016'] - df_analysis['Obama\r\n2008']

# Summary statistics for this new column
print('Change in Democratic vote share (2008 → 2016):')
print(df_analysis['Dem_change'].describe())

In [ ]:
# Which districts saw the largest swings toward Democrats?
df_analysis.sort_values('Dem_change', ascending=False)[['CD', 'Incumbent', 'Dem_change']].head(10)

In [ ]:
# Which districts swung most toward Republicans?
df_analysis.sort_values('Dem_change', ascending=True)[['CD', 'Incumbent', 'Dem_change']].head(10)

In [ ]:
# Visualize the distribution of the swing
plt.figure()
df_analysis['Dem_change'].plot(kind='hist', bins=40, edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='No change')
plt.xlabel('Change in Democratic vote share (2008 → 2016, %)')
plt.title('How did districts shift between 2008 and 2016?')
plt.legend()
plt.show()

**Final exercise:** Repeat the analysis above for the Republican side — compute the change in Republican vote share from 2008 to 2016, then find the 5 districts with the largest Republican swings and the 5 districts with the largest Democratic swings. Plot a histogram. Does the distribution look like what you'd expect given the Democratic one above?

In [ ]:
df_analysis['Rep_change'] = df_analysis['Trump\r\n2016'] - df_analysis['Romney\r\n2012']

print('Change in Republican vote share (2012 → 2016):')
print(df_analysis['Rep_change'].describe())

print('\n5 districts with largest Republican swing (toward Trump):')
print(df_analysis.sort_values('Rep_change', ascending=False)[['CD', 'Incumbent', 'Rep_change']].head(5).to_string())

print('\n5 districts with largest Democratic swing (away from Trump):')
print(df_analysis.sort_values('Rep_change', ascending=True)[['CD', 'Incumbent', 'Rep_change']].head(5).to_string())

plt.figure()
df_analysis['Rep_change'].plot(kind='hist', bins=40, edgecolor='black')
plt.axvline(0, color='red', linestyle='--', label='No change')
plt.xlabel('Change in Republican vote share (2012 → 2016, %)')
plt.title('How did districts shift between 2012 and 2016?')
plt.legend()
plt.show()
# The Republican distribution is roughly the mirror image of the Democratic one --
# Trump gained in most districts where Clinton lost, and vice versa,
# as expected in a two-party system where votes are roughly conserved.